In [1]:
import numpy as np

# 先运行并载入 basis_func_with_kinetic.ipynb 中定义的函数（primitive_fcc, reciprocal_vectors, generate_G_vectors 等）
%run basis_func_with_kinetic.ipynb

HARTREE_TO_EV = 27.211386

def compute_hartree_energy_from_nG(nG, Gvecs, volume):
    """根据 n(G) 和对应的 G 向量计算 Hartree 能（原子单位：Hartree）。"""
    nG_flat = nG.ravel()
    G_flat = Gvecs.reshape((-1, 3))

    norms = np.linalg.norm(G_flat, axis=1)
    mask = norms > 1e-12
    G_nonzero = G_flat[mask]
    nG_nonzero = nG_flat[mask]

    prefac = 2.0 * np.pi / volume
    E_H = prefac * np.sum(np.abs(nG_nonzero)**2 / (np.linalg.norm(G_nonzero, axis=1)**2))
    return E_H

def atomic_orbital_gaussian(r, center, alpha, l=0, m=0):
    """
    高斯型原子轨道在实空间的值。
    r: 坐标数组 (..., 3)
    center: 轨道中心坐标 (3,)
    alpha: 高斯指数
    l, m: 角量子数（目前只实现s轨道）
    """
    r_rel = r - center
    r2 = np.sum(r_rel**2, axis=-1)
    
    # s轨道
    if l == 0 and m == 0:
        return (2*alpha/np.pi)**0.75 * np.exp(-alpha * r2)
    else:
        # 可以扩展p,d轨道
        raise NotImplementedError("Currently only s orbitals are implemented")

def generate_lcao_wavefunctions(atomic_positions, atomic_types, a_vecs, Nk=32):
    """
    使用LCAO方法生成初始波函数。
    atomic_positions: 原子位置列表 (frac_coords)
    atomic_types: 原子类型列表
    a_vecs: 实空间晶格矢量
    Nk: 网格点数
    """
    # 原子轨道参数（高斯指数）
    atomic_orbital_params = {
        'Al': [0.5, 1.0, 2.0],  # 三个s轨道的指数
        'Si': [0.6, 1.2, 2.5],
        'H': [1.0]
    }
    
    us = np.arange(Nk) / Nk
    U, V, W = np.meshgrid(us, us, us, indexing='ij')
    r_grid = (U[...,None] * a_vecs[0] + V[...,None] * a_vecs[1] + W[...,None] * a_vecs[2])
    
    wavefunctions = []
    
    for atom_pos, atom_type in zip(atomic_positions, atomic_types):
        # 将分数坐标转换为笛卡尔坐标
        center_cart = atom_pos[0] * a_vecs[0] + atom_pos[1] * a_vecs[1] + atom_pos[2] * a_vecs[2]
        
        if atom_type in atomic_orbital_params:
            alphas = atomic_orbital_params[atom_type]
            for i, alpha in enumerate(alphas):
                # 生成原子轨道
                orbital = atomic_orbital_gaussian(r_grid, center_cart, alpha)
                wavefunctions.append(orbital)
    
    return wavefunctions, r_grid

def orthogonalize_wavefunctions(wavefunctions, r_grid, a_vecs):
    """
    对波函数进行正交归一化。
    """
    Nk = r_grid.shape[0]
    dV = np.dot(a_vecs[0], np.cross(a_vecs[1], a_vecs[2])) / (Nk**3)
    
    orthogonalized = []
    
    for i, psi_i in enumerate(wavefunctions):
        # 减去与之前所有波函数重叠的部分
        psi_ortho = psi_i.copy()
        
        for j in range(i):
            overlap = np.sum(np.conj(orthogonalized[j]) * psi_i) * dV
            psi_ortho = psi_ortho - overlap * orthogonalized[j]
        
        # 归一化
        norm = np.sqrt(np.sum(np.abs(psi_ortho)**2) * dV)
        if norm > 1e-12:
            psi_ortho = psi_ortho / norm
            orthogonalized.append(psi_ortho)
    
    return orthogonalized

def build_lcao_charge_density(wavefunctions, occupations=None):
    """
    从LCAO波函数构建电荷密度。
    wavefunctions: 正交归一化的波函数列表
    occupations: 每个轨道的占据数，如果为None则默认占据前N个轨道
    """
    n_bands = len(wavefunctions)
    if occupations is None:
        # 默认：每个带占据2个电子（考虑自旋）
        occupations = [2.0] * n_bands
    
    # 初始化电荷密度
    charge_density = np.zeros_like(wavefunctions[0], dtype=float)
    
    for i, psi in enumerate(wavefunctions):
        if i < len(occupations):
            charge_density += occupations[i] * np.abs(psi)**2
    
    return charge_density

def density_grid_to_reciprocal(n_r, b_vecs):
    """从实空间密度网格生成 n(G) 与对应的 G 网格，便于后续能量计算。"""
    if n_r.ndim != 3 or n_r.shape[0] != n_r.shape[1] or n_r.shape[1] != n_r.shape[2]:
        raise ValueError('n_r must be a cubic grid with shape (Nk, Nk, Nk).')
    Nk = n_r.shape[0]
    nG_fft = np.fft.fftn(n_r)
    nG = nG_fft / nG_fft.size
    freq = np.fft.fftfreq(Nk) * Nk
    mx, my, mz = np.meshgrid(freq, freq, freq, indexing='ij')
    Ggrid = (mx[..., None] * b_vecs[0] + my[..., None] * b_vecs[1] + mz[..., None] * b_vecs[2]) / Nk
    return nG, Ggrid

def compute_hartree_potential_from_charge_density(n_r, b_vecs, volume):
    """
    从电荷密度计算Hartree势。
    """
    nG, Ggrid = density_grid_to_reciprocal(n_r, b_vecs)
    
    # Hartree势在倒空间：V_H(G) = 4π n(G) / |G|^2 (G ≠ 0)
    nG_flat = nG.ravel()
    G_flat = Ggrid.reshape((-1, 3))
    norms = np.linalg.norm(G_flat, axis=1)
    
    VH_G = np.zeros_like(nG_flat, dtype=complex)
    mask = norms > 1e-12
    VH_G[mask] = 4.0 * np.pi * nG_flat[mask] / (norms[mask]**2)
    
    # 转换回实空间
    VH_G_reshaped = VH_G.reshape(nG.shape)
    VH_r = np.fft.ifftn(VH_G_reshaped * nG.size)  # 乘以N^3来补偿FFT归一化
    
    return np.real(VH_r)

# ---------------- 主程序：使用LCAO方法生成电荷密度并计算Hartree能 ----------------
def main_lcao_hartree():
    # 设置体系：FCC铝，2个原子
    a0 = 4.05  # Å (Al晶格常数)
    a_vecs = primitive_fcc(a0)
    Omega = np.dot(a_vecs[0], np.cross(a_vecs[1], a_vecs[2]))
    
    # 原子位置（分数坐标）
    atomic_positions = [
        [0.0, 0.0, 0.0],  # 第一个原子
        [0.5, 0.5, 0.5]   # 第二个原子
    ]
    atomic_types = ['Al', 'Al']
    
    Nk = 32
    
    print("=== LCAO方法计算Hartree能 ===")
    print(f"晶格常数: {a0} Å")
    print(f"单胞体积: {Omega:.6f} bohr^3")
    print(f"原子数: {len(atomic_positions)}")
    print(f"网格点数: {Nk}^3")
    
    # 1. 生成LCAO波函数
    print("\n1. 生成LCAO波函数...")
    wavefunctions, r_grid = generate_lcao_wavefunctions(atomic_positions, atomic_types, a_vecs, Nk)
    print(f"生成的原子轨道数: {len(wavefunctions)}")
    
    # 2. 正交归一化波函数
    print("2. 正交归一化波函数...")
    wavefunctions_ortho = orthogonalize_wavefunctions(wavefunctions, r_grid, a_vecs)
    print(f"正交归一化后的波函数数: {len(wavefunctions_ortho)}")
    
    # 3. 构建电荷密度
    print("3. 构建电荷密度...")
    # 对于Al，假设每个原子贡献3个价电子
    n_electrons = len(atomic_positions) * 3
    occupations = [2.0] * (n_electrons // 2)  # 每个轨道2个电子（闭壳层）
    charge_density = build_lcao_charge_density(wavefunctions_ortho, occupations)
    
    # 检查总电子数
    dV = Omega / (Nk**3)
    total_electrons = np.sum(charge_density) * dV
    print(f"总电子数: {total_electrons:.6f} (目标: {n_electrons})")
    
    # 4. 计算倒空间格矢
    b_vecs, _ = reciprocal_vectors(a_vecs)
    
    # 5. 计算Hartree能
    print("4. 计算Hartree能...")
    nG, Ggrid = density_grid_to_reciprocal(charge_density, b_vecs)
    E_H_ha = compute_hartree_energy_from_nG(nG, Ggrid, Omega)
    
    print(f"\n结果:")
    print(f"Hartree能 (Hartree): {E_H_ha:.8f}")
    print(f"Hartree能 (eV): {E_H_ha * HARTREE_TO_EV:.6f} eV")
    
    # 6. 可选：计算Hartree势
    print("\n5. 计算Hartree势...")
    VH_r = compute_hartree_potential_from_charge_density(charge_density, b_vecs, Omega)
    print(f"Hartree势范围: {np.min(VH_r):.6f} 到 {np.max(VH_r):.6f} Hartree")
    
    return {
        'charge_density': charge_density,
        'hartree_energy': E_H_ha,
        'hartree_potential': VH_r,
        'wavefunctions': wavefunctions_ortho,
        'volume': Omega
    }

# 运行示例
if __name__ == '__main__':
    results = main_lcao_hartree()

=== LCAO方法计算Hartree能 ===
晶格常数: 4.05 Å
单胞体积: 112.073152 bohr^3
原子数: 2
网格点数: 32^3

1. 生成LCAO波函数...
生成的原子轨道数: 6
2. 正交归一化波函数...
正交归一化后的波函数数: 6
3. 构建电荷密度...
总电子数: 6.000000 (目标: 6)
4. 计算Hartree能...

结果:
Hartree能 (Hartree): 1.48426933
Hartree能 (eV): 40.389026 eV

5. 计算Hartree势...
Hartree势范围: -805.171369 到 11294.110194 Hartree
